# Petryk

A hyperactive kid who never stops listening, never forgets, and always writes back.

This notebook shows how to send data to Petryk's brain and read it back. No fixed schema — send whatever you want.

In [ ]:
import requests

API = "https://yh9fp9463n.us-east-1.awsapprunner.com"

## Write to Petryk

Send any JSON to `POST /data`. The only required field is `email` — everything else is up to you.

In [ ]:
response = requests.post(f"{API}/data", json={
    "email": "you@example.com",
    "name": "my first entry",
    "value": 42,
    "tags": ["test", "notebook"],
    "notes": "Petryk remembers everything",
})

item = response.json()
print(item)

## Read it back

Grab a specific item by ID.

In [ ]:
item_id = item["id"]

response = requests.get(f"{API}/data/{item_id}")
response.json()

## See everything Petryk knows

Fetch all items in the database.

In [ ]:
response = requests.get(f"{API}/data")
all_items = response.json()

print(f"Petryk has {len(all_items)} memories")
for i in all_items[:5]:
    print(f"  {i['id'][:8]}... — {list(i.keys())}")

## Update an item

Correct something Petryk got wrong.

In [ ]:
response = requests.put(f"{API}/data/{item_id}", json={
    "email": "you@example.com",
    "name": "updated entry",
    "value": 99,
    "tags": ["test", "notebook", "updated"],
})

response.json()

## Delete an item

Tell Petryk to forget.

In [ ]:
response = requests.delete(f"{API}/data/{item_id}")
response.json()

## Bulk write

Feed Petryk a bunch of data at once.

In [ ]:
entries = [
    {"email": "you@example.com", "type": "metric", "name": "temperature", "value": 72.5},
    {"email": "you@example.com", "type": "metric", "name": "humidity", "value": 45},
    {"email": "you@example.com", "type": "note", "content": "Petryk is getting smarter"},
]

for entry in entries:
    r = requests.post(f"{API}/data", json=entry)
    print(f"  Stored: {r.json()['id'][:8]}...")

print(f"\nFed Petryk {len(entries)} new memories")

## Gemini Vision — Petryk Sees

Use Google's Gemini API to describe an image, then feed the result straight into Petryk's brain.

In [ ]:
!pip install google-genai Pillow

In [ ]:
import os
from google import genai
from PIL import Image

# Set your Gemini API key (or set GEMINI_API_KEY env var)
os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY"

client = genai.Client()

### Describe an image with Gemini

Point it at any image file. Gemini will describe it, then we send that description to Petryk.

In [ ]:
# Open any image
image = Image.open("your_image.jpg")

# Ask Gemini to describe it
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=["Describe this image in detail", image],
)

description = response.text
print(description)

### Feed the description to Petryk

Now Petryk knows what's in the image — without ever seeing it himself.

In [ ]:
# Send Gemini's description to Petryk
response = requests.post(f"{API}/data", json={
    "email": "you@example.com",
    "type": "vision",
    "source": "your_image.jpg",
    "model": "gemini-2.5-flash",
    "description": description,
})

print("Petryk remembers:", response.json()["id"])